In [1]:
from pathlib import Path
import json
import pandas as pd
import librosa

In [2]:
FUSA_SERVER="https://api.labacam.org"
API_KEY="zIkYk2VwBiC762O4yFSd"

ENDPOINT="predictions"
MODEL="PANN-tag-5s"

URI = f"{FUSA_SERVER}/{ENDPOINT}?model={MODEL}/"

print(URI)

https://api.labacam.org/predictions?model=PANN-tag-5s/


In [3]:
threshold = 0.01

In [4]:
def filter_predictions(categories_dict, threshold):
    filtered_dict = {}
    for key, values in categories_dict.items():
        values_list = []
        for value in values:
            if value['probability'] > threshold:
                values_list.append(value)
            filtered_dict.update({key:values_list})
    return filtered_dict

def calculate_metrics(filtered_predictions, audio_duration):
    df = pd.DataFrame([], columns  = ['event', 'duration_ratio', 'avg_acc'])
    class_tags, duration_ratio, avg_acc = [], [], []
    for class_tags in filtered_predictions:
        avg_acc = 0
        tags = filtered_predictions[class_tags]
        c = len(tags)
        if c > 0:
            for tag in tags:
                avg_acc += tag['probability']
            duration_ratio = (c * 5) / audio_duration
            avg_acc = avg_acc / c
            d = {'event' : class_tags, 'duration_ratio': round(duration_ratio, 3), 'avg_acc': round(avg_acc, 3)}
            output = pd.DataFrame([d])
            df = pd.concat([df, output], ignore_index=True)
    df = df.sort_values(by='duration_ratio', ascending=False).head(5)
    return df

def get_audio_duration(audio_path):
    return librosa.get_duration(filename=audio_path)

In [8]:
audio_name = "/home/rafael-hayde/Desktop/fusa/fusa-net-experiments/datasets/VitGlobal-ROADS/audios/ERUIDO1994_20231031011217.mp3"
response = !curl -s -X POST {URI} -H  "accept: application/json" -H  "X-Api-Key:{API_KEY}" -H  "Content-Type: multipart/form-data" -F "file=@{audio_name};type=audio/mp3"

try:
    categories_dict = json.loads(response[0])['categories']
    filtered_predictions = filter_predictions(categories_dict, threshold)
    audio_duration = get_audio_duration(audio_name)
    classes_metrics = calculate_metrics(filtered_predictions, audio_duration)
    audio_path = Path(audio_name)
    classes_metrics['filename'] = str(Path(audio_path))
except Exception as e:
    print(e, audio_name)

list index out of range /home/rafael-hayde/Desktop/fusa/fusa-net-experiments/datasets/VitGlobal-ROADS/audios/ERUIDO1994_20231031011217.mp3


In [6]:
classes_metrics

NameError: name 'classes_metrics' is not defined